In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [5]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.55, 1.6, 1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.5, 0.6, 0.7, 0.9)]
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.058126                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.040637
0.000750      2.0  0.078911                  2.0 -0.021157
0.001052      3.0  0.141464                  3.0  0.025424
0.001476      4.0  0.211606                  4.0  0.079074
0.002070      5.0  0.287575                  5.0  0.139409
0.002904      6.0  0.368265                  6.0  0.206445
0.004074      7.0  0.451700                  7.0  0.281689
0.005714      8.0  0.535058                  8.0  0.364688
0.008015      9.0  0.614832                  9.0  0.453147
0.011243     10.0  0.686668                 10.0  0.544765
0.015771     11.0  0.745775                 11.0  0.636146
0.022122     12.0  0.787977                 12.0  0.722446
0.031031     13.0  0.810552                 13.0  0.796157
0.043528     14.0  0.792570                 14.0  0.830016
0.061057     15.0  0.720908                 15.0  0.798648
0.085645     16.0  0.627889                 16.0  0.716484
0.120136     17.0  0.541781                 17.0  0.606414
0.168516     18.0  0.470042                 18.0  0.487522
0.236378     19.0  0.417906                 19.0  0.388063
0.331549     20.0  0.392114                 20.0  0.337132
0.465100     21.0  0.408602                 21.0  0.358667
0.652400     22.0  0.365494                 22.0  0.340307
0.915100     23.0  0.257275                 23.0  0.255023
1.283650     24.0  0.184958                 24.0  0.193586
1.800600     25.0  0.143412                 25.0  0.152562
2.525700     26.0  0.112290                 26.0  0.118788
3.542800     27.0  0.084953                 27.0  0.091667
4.969550     28.0  0.066108                 28.0  0.074811
6.970850     29.0  0.057551                 29.0  0.068820
9.778100     30.0  0.050907                 30.0  0.062098
13.715850    31.0  0.042543                 31.0  0.050258
19.239350    32.0  0.033040                 32.0  0.034974
26.987250    33.0  0.028598                 33.0  0.025401
37.855300    34.0  0.029534                 34.0  0.023451
53.100050    35.0  0.029642                 35.0  0.024657
73.887500    36.0  0.029361                 36.0  0.028606
97.662500    37.0  0.028983                 37.0  0.031319
121.437500   38.0  0.026578                 38.0  0.029925
145.212500   39.0  0.023138                 39.0  0.026204
168.987500   40.0  0.020196                 40.0  0.022387
192.762500   41.0  0.017212                 41.0  0.018328
216.537500   42.0  0.013987                 42.0  0.014008
240.312500   43.0  0.010304                 43.0  0.009153
264.087500   44.0  0.005607                 44.0  0.003290
287.862500   45.0 -0.000864                 45.0 -0.004100
311.637500   46.0 -0.003061                 46.0 -0.006890
335.412500   47.0  0.002888                 47.0 -0.001171
359.187500   48.0  0.010780                 48.0  0.006820
382.962500   49.0  0.017491                 49.0  0.013764
406.737500   50.0  0.023930                 50.0  0.020305
430.512500   51.0  0.030396                 51.0  0.026776
454.287500   52.0  0.037012                 52.0  0.033386
478.062500   53.0  0.043814                 53.0  0.040284
501.837500   54.0  0.050766                 54.0  0.047451
525.612500   55.0  0.057778                 55.0  0.054701
549.387500   56.0  0.064692                 56.0  0.062106
573.162500   57.0  0.071330                 57.0  0.069445
596.937500   58.0  0.077488                 58.0  0.076448
620.712500   59.0  0.082980                 59.0  0.082837
644.487500   60.0  0.087672                 60.0  0.088363
668.262500   61.0  0.091507                 61.0  0.092951
692.037500   62.0  0.094548                 62.0  0.096808
715.812500   63.0  0.096931                 63.0  0.099983
739.587500   64.0  0.098874                 64.0  0.102688
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -16.337494  0.000000 -16.337494        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -16.241721  0.000001   
6.244000e-04 2     -16.337526  0.000036 -16.337490 -16.241749  0.000026   
8.759000e-04 3     -16.337538  0.000051 -16.337488 -16.241761  0.000037   
1.228600e-03 4     -16.337556  0.000074 -16.337482 -16.241776  0.000054   
1.723400e-03 5     -16.337579  0.000110 -16.337469 -16.241798  0.000080   
2.417400e-03 6     -16.337612  0.000166 -16.337446 -16.241827  0.000121   
3.390900e-03 7     -16.337654  0.000251 -16.337403 -16.241866  0.000184   
4.756500e-03 8     -16.337711  0.000381 -16.337330 -16.241918  0.000281   
6.672000e-03 9     -16.337785  0.000576 -16.337209 -16.241986  0.000432   
9.358900e-03 10    -16.337879  0.000866 -16.337013 -16.242075  0.000665   
1.312780e-02 11    -16.337998  0.001291 -16.336706 -16.242186  0.001019   
1.841450e-02 12    -16.338141  0.001902 -16.336239 -16.242321  0.001553   
2.583020e-02 13    -16.338308  0.002760 -16.335547 -16.242479  0.002346   
3.623230e-02 14    -16.338486  0.003938 -16.334548 -16.242650  0.003497   
5.082340e-02 15    -16.338665  0.005486 -16.333179 -16.242822  0.005104   
7.129060e-02 16    -16.338842  0.007412 -16.331431 -16.242994  0.007213   
1.000000e-01 17    -16.339015  0.009720 -16.329295 -16.243159  0.009815   
1.402710e-01 18    -16.339153  0.012442 -16.326711 -16.243287  0.012836   
1.967600e-01 19    -16.339198  0.015632 -16.323566 -16.243322  0.016134   
2.759970e-01 20    -16.339033  0.019390 -16.319643 -16.243153  0.019607   
3.871000e-01 21    -16.338389  0.023907 -16.314482 -16.242515  0.023407   
5.431000e-01 22    -16.336412  0.029481 -16.306932 -16.240526  0.028046   
7.617000e-01 23    -16.332806  0.035339 -16.297467 -16.236804  0.033137   
1.068500e+00 24    -16.328615  0.040497 -16.288117 -16.232390  0.037993   
1.498800e+00 25    -16.324330  0.045641 -16.278690 -16.227798  0.043269   
2.102400e+00 26    -16.319972  0.051536 -16.268436 -16.223041  0.049421   
2.949000e+00 27    -16.315643  0.058469 -16.257175 -16.218226  0.056521   
4.136600e+00 28    -16.311799  0.066576 -16.245223 -16.213760  0.064952   
5.802500e+00 29    -16.308809  0.076631 -16.232178 -16.209947  0.075903   
8.139200e+00 30    -16.306398  0.090151 -16.216248 -16.206440  0.091447   
1.141700e+01 31    -16.304562  0.108081 -16.196482 -16.203227  0.112349   
1.601470e+01 32    -16.304237  0.130925 -16.173311 -16.201371  0.137867   
2.246400e+01 33    -16.307938  0.159868 -16.148070 -16.203729  0.166947   
3.151050e+01 34    -16.317472  0.200048 -16.117424 -16.212293  0.202734   
4.420010e+01 35    -16.332377  0.259348 -16.073030 -16.226055  0.251752   
6.200000e+01 36    -16.353986  0.343457 -16.010528 -16.246441  0.324133   
8.577500e+01 37    -16.383177  0.455339 -15.927838 -16.274728  0.432993   
1.095500e+02 38    -16.412615  0.566403 -15.846212 -16.304041  0.550521   
1.333250e+02 39    -16.444309  0.672950 -15.771359 -16.336367  0.667135   
1.571000e+02 40    -16.480793  0.774597 -15.706195 -16.374099  0.778672   
1.808750e+02 41    -16.522585  0.873269 -15.649317 -16.417798  0.885428   
2.046500e+02 42    -16.570300  0.969458 -15.600842 -16.468280  0.987532   
2.284250e+02 43    -16.624804  1.063352 -15.561451 -16.527133  1.085840   
2.522000e+02 44    -16.687507  1.155074 -15.532433 -16.596105  1.180593   
2.759750e+02 45    -16.761431  1.244790 -15.516641 -16.678317  1.272072   
2.997500e+02 46    -16.853504  1.334430 -15.519075 -16.780113  1.362319   
3.235250e+02 47    -16.964377  1.436681 -15.527696 -16.901011  1.463809   
3.473000e+02 48    -17.084967  1.565404 -15.519563 -17.031086  1.590585   
3.710750e+02 49    -17.209306  1.720105 -15.489202 -17.164360  1.743070   
3.948500e+02 50    -17.336370  1.896429 -15.439941 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')